<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/re_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import pandas as pd
import re

In [50]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

In [51]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.sample(25)


,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
211,Indianapolis Colts,14,DST,218.8,8.641063,8.545135,8.200903,8.263049,7.893689,9.135791,...,8.735751,7.902211,9.304558,8.099938,7.936804,7.820451,0.000000,7.291624,8.202004,8.641865
139,Curtis Samuel,12,WR,143.6,9.434562,11.655392,10.063156,8.485976,9.877296,9.923330,...,10.261414,9.146438,8.641471,8.294507,0.000000,9.462272,8.444945,10.235057,10.237953,8.958119
49,C.J. Stroud,14,QB,48.6,20.599138,18.678996,16.782974,19.616728,15.301311,19.481149,...,20.977016,15.597960,16.040716,20.177189,20.000510,17.263723,0.000000,17.196018,17.285094,20.538684
177,Bo Nix,14,QB,184.6,12.881534,11.334501,11.422018,12.816542,12.604398,13.859211,...,14.626992,12.975715,11.505387,15.155551,13.136211,13.330816,0.000000,10.647744,12.255470,14.841261
85,Austin Ekeler,14,RB,91.0,12.819334,9.300399,11.662773,12.452305,9.137017,10.466836,...,11.791309,11.961784,10.655285,10.246457,14.060515,10.581215,0.000000,7.892527,11.553521,11.977099
12,Jahmyr Gibbs,5,RB,12.8,21.458336,19.709543,16.389694,23.408326,0.000000,18.905980,...,17.849042,19.743098,23.563878,21.443044,22.641772,21.236795,19.558414,18.231262,20.827388,20.813992
191,Derek Carr,12,QB,218.0,10.848194,12.446872,12.878876,13.427376,12.420153,14.301440,...,11.280484,12.440217,14.215001,14.930984,0.000000,11.715265,12.450125,11.229192,10.808291,12.396209
199,Hunter Henry,14,TE,193.0,10.237234,9.461884,8.852624,8.891102,10.168020,9.210826,...,7.674788,8.747138,8.775280,10.273822,8.065103,8.765299,0.000000,7.141475,8.941864,9.195899
1,CeeDee Lamb,7,WR,2.6,23.167916,18.543061,21.848593,24.200848,19.747865,20.305863,...,23.034764,19.287060,28.397460,23.543259,20.558213,20.883968,22.974291,26.539097,20.938460,23.452845
151,Tyler Allgeier,12,RB,156.8,9.119185,9.127955,9.613333,8.015742,7.813149,8.980272,...,9.162856,8.947186,9.588528,10.415910,0.000000,8.729690,9.613688,11.064783,8.539911,7.922384


In [53]:
week_cols = [col for col in f.columns if col.startswith("Week_")]

# Dictionary: f_dict[player][week] = projection
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}


In [9]:
#!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 38.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [54]:
from mip import Model, BINARY, maximize, xsum

# Initialisierung des Modells
model = Model("FantasyFootball", sense=maximize)

# Datenstrukturen (Beispielwerte)
players = df[['Player', 'POS']]
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks = list(range(1, 18))
gamma = {"QB": 2, "RB": 5, "WR": 6, "TE": 2, "K": 1, "DST": 2}
pos_limit = {"QB": 1, "RB": 2, "WR":2, "TE": 1, "K": 1, "DST": 1}

# Entscheidungsvariablen
y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players}
x = {(i,t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

# Zielfunktion (Beispielgewichte)
model.objective = (
    1.0 * xsum(f[i][t] * x[i,t] for i in players for t in weeks) +
    100.0 * xsum(z[t] for t in range(1,16)) +
    150.0 * xsum(z[t] for t in range(16,18))
)

# Constraints
for j in positions:
    # Mindestanzahl gedrafteter Spieler pro Position (1c)
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]

    # Positionslimits pro Woche (1d)
    for t in weeks:
        model += xsum(x[i,t] for i in players if pos[i] == j) <= pos_limit[j]

# Logische Verknüpfung Draft ↔ Aufstellung (1e)
for i in players:
    for t in weeks:
        model += x[i,t] <= y[i]

# Gewinnbedingung (1f)
for t in weeks:
    model += z[t] <= xsum(f[i][t] * x[i,t] for i in players) / beta[t]

# Bereits gedraftete Spieler (1g-1h)
for i in dm_players:
    model += y[i] == 1
for i in opp_players:
    model += y[i] == 0

# Optimierung
model.optimize()


TypeError: Can not multiply with type <class 'str'>